In [1]:
%pylab widget
from copy import deepcopy
from reed_up_downstream_dyn import ReedSimulation, calc_fixed_point
from json_object import JSONObject
from pypevoc.PVAnalysis import PV
from pypevoc.Heterodyne import HeterodyneHarmonic
from pypevoc.SoundUtils import FuncWind
from scipy.optimize import fsolve

Populating the interactive namespace from numpy and matplotlib


In [89]:
with open('reed_simulation_dyn.json') as f:
    js = JSONObject(f)
js['environment/blowing pressure/value']=2200
#js['environment/perturbation time']=0.01
js['simulation/duration']=.4
js['tracts/vocal/frequency independent losses']=0.1
js['tracts/vocal/elements/0/length']=0.15
js['tracts/vocal/elements/0/radius']=0.015
js['tracts/vocal/elements/0/loss multiplier']=5
js['environment/frequency dependent losses']=True
js['environment/vocal tract enabled']=True
js['environment/reed/dynamic']=True
js['environment/reed/quality factor']=1.5
js['environment/reed/resonance frequency']=1500

js.to_python()
    


{'description': 'Reed simulation with vocal tract and reed dynamics',
 'version': '20200311',
 'simulation': {'sample rate': 48000, 'duration': 0.4, 'callback every': 1024},
 'environment': {'acoustic': {'_prefer': True,
   'speed of sound': 346.7492013525034,
   'density': 1.2,
   'viscosity': 1.884e-05},
  'physical': {'atmospheric pressure': 101500,
   'temperature': 36.5,
   'humidity': 100},
  'blowing pressure': {'_comment': "blowing rpessure is 'value' from start if ramp not enabled",
   'value': 2200,
   'ramp type': 'exponential',
   'ramp enabled': False},
  'reed': {'_comment': "reed moves instantaneously if 'dynamic' is false, that means res. freq and q are not taken into account",
   'stiffness': 500000000.0,
   'resonance frequency': 1500,
   'quality factor': 1.5,
   'rest opening': 1e-05,
   'dynamic': True},
  'noise': {'enabled': False, 'type': 'white', 'scale': 0.005},
  'frequency dependent losses': True,
  'vocal tract enabled': True},
 'perturbation': {'factor': 1

In [90]:

sim = ReedSimulation()
sim.from_json(js)
sim.pert=1.0
sim.simulate(4000)


figure()
p_b = sim.p_in + sim.p_out;
p_vt = sim.p_vt_in + sim.p_vt_out;

u = (sim.p_out - sim.p_in)/sim.zc_b;
u_sg = -(sim.p_vt_out - sim.p_vt_in)/sim.zc_vt

a = sim.a
lns=plot(p_b)

p_in_avg, t_avg = FuncWind(np.sum,sim.p_in)
p_out_avg, t_avg = FuncWind(np.sum,sim.p_out)
plot(t_avg,p_in_avg+p_out_avg,color=lns[0].get_color(),alpha=.5)
lns=plot(p_vt)

p_vt_in_avg, t_avg = FuncWind(np.sum,sim.p_vt_in)
p_vt_out_avg, t_avg = FuncWind(np.sum,sim.p_vt_out)
plot(t_avg,p_vt_in_avg+p_vt_out_avg,color=lns[0].get_color(),alpha=.5)



applied perturbation at sample 97


/Users/goios/Envs/unsw/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [91]:
figure()
plot(sim.a)

/Users/goios/Envs/unsw/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [92]:
sim = ReedSimulation()
sim.from_json(js)

dp_per_samp = 0.5
n_samp=48000
frac = dp_per_samp/js['environment/blowing pressure/value']

pert_pressure = 2100
pert_ampl = 1.01

sim.simulation_init(pert=False)
sim.p_blow=0.0
sim.pert=False
p_blow = []
while sim.samp_no < n_samp:
    sim.simulation_tick(reverse=False)
    if sim.p_blow<pert_pressure:
        sim.p_blow+=(js['environment/blowing pressure/value']-sim.p_blow)*frac
    else:
        if pert_ampl:
            sim.a0*=pert_ampl
            pert_ampl=False
            pert_samp = sim.samp_no
    p_blow.append(sim.p_blow)
sim.finalize()
p_blow= np.array(p_blow)

In [93]:
figure()
p_b = sim.p_in + sim.p_out;
p_vt = sim.p_vt_in + sim.p_vt_out;

u = (sim.p_out - sim.p_in)/sim.zc_b;
u_sg = -(sim.p_vt_out - sim.p_vt_in)/sim.zc_vt

a = sim.a
lns=plot(p_b)
p_in_avg, t_avg = FuncWind(np.sum,sim.p_in)
p_out_avg, t_avg = FuncWind(np.sum,sim.p_out)
plot(t_avg,p_in_avg+p_out_avg,color=lns[0].get_color(),alpha=.5)

lns=plot(np.array(p_blow)+p_vt)
p_in_avg, t_avg = FuncWind(np.sum,sim.p_vt_in)
p_out_avg, t_avg = FuncWind(np.sum,sim.p_vt_out)
plot(t_avg,p_in_avg+p_out_avg,color=lns[0].get_color(),alpha=.5)

p_shut = sim.k*sim.a0
axhline(p_shut,color='k',ls='--',alpha=.5)
axhline(p_shut/3,color='k',ls='--',alpha=.5)
samp_thr = np.flatnonzero(p_blow>p_shut/3)[0]
axvline(samp_thr,color='k',alpha=.5,ls='--')
axvline(pert_samp,color='r',alpha=.5,ls='--')

/Users/goios/Envs/unsw/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [94]:
f0=1/(sim.tracts['bore'].total_delay/sim.sr*2)
hhb = HeterodyneHarmonic(p_b,sr=sim.sr,nwind=1024,nhop=256,f=f0)
hhv = HeterodyneHarmonic(p_vt,sr=sim.sr,nwind=1024,nhop=256,f=f0)

In [95]:
figure()
semilogy(hhb.t,np.abs(hhb.camp[:,0]))
semilogy(hhv.t,np.abs(hhv.camp[:,0]))

/Users/goios/Envs/unsw/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [96]:
sim.tracts['vocal'].tubes